# TP – Analyse de la qualité de l'air
## Etape 3 – Nettoyage avancé Pandas

### Imports

In [16]:
import pandas as pd
import numpy as np

### Charger `weather_raw.csv` avec Pandas:

In [2]:
data_file_path = "../data/weather_raw.csv"
df_weather = pd.read_csv(data_file_path)
df_weather.head()

,city,timestamp,temperature_c,humidity_pct,wind_speed_kmh,precipitation_mm,weather_condition
0,Grenoble,2024-03-12 09:00:00,14.8,74.6,24.8,2.4,pluvieux
1,Strasbourg,05/04/2024 21:00,10.8,78.4,5.7,0.0,neigeux
2,Marseille,28/01/2024 23:00,13.3,60.4,45.7,3.5,pluvieux
3,Bordeaux,29/01/2024 12:00,1.3,84.1,43.3,0.0,brumeux
4,Marseille,2024-01-15 00:00:00,13.3,87.4,21.7,6.0,orageux


### Schema de la df_weather:

In [3]:
df_weather.info()

<class 'pandas.DataFrame'>
RangeIndex: 42172 entries, 0 to 42171
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   city               42172 non-null  str    
 1   timestamp          42172 non-null  str    
 2   temperature_c      41724 non-null  str    
 3   humidity_pct       41760 non-null  float64
 4   wind_speed_kmh     42172 non-null  float64
 5   precipitation_mm   42172 non-null  float64
 6   weather_condition  41310 non-null  str    
dtypes: float64(3), str(4)
memory usage: 2.3 MB


### Identifier et traiter les valeurs manquantes :
  - Interpolation linéaire pour température et humidité:

1) Les colonnes qui ont des NaN :

In [4]:
df_weather.isna().sum()

city                   0
timestamp              0
temperature_c        448
humidity_pct         412
wind_speed_kmh         0
precipitation_mm       0
weather_condition    862
dtype: int64

=> Il y'a des NaN dans **temperature_c**, **humidity_pct** et weather_condition.    

2) L’interpolation linéaire a du sens dans l’ordre temporel: donc on s’assure que timestamp est bien en datetime et on trie:

In [5]:
df_weather["timestamp"] = pd.to_datetime(df_weather["timestamp"], errors="coerce")
df_weather = df_weather.sort_values("timestamp")
df_weather.head()

,city,timestamp,temperature_c,humidity_pct,wind_speed_kmh,precipitation_mm,weather_condition
6149,Bordeaux,2024-01-01 00:00:00,"11,1",64.7,1.0,4.1,pluvieux
26579,Rouen,2024-01-01 01:00:00,0.0,41.7,17.5,0.0,ensoleille
33993,Strasbourg,2024-01-01 01:00:00,10.0,65.0,43.2,0.0,orageux
35309,Lille,2024-01-01 02:00:00,10.0,64.1,20.8,0.0,nuageux
3864,Grenoble,2024-01-01 03:00:00,6.2,72.9,5.9,0.0,pluvieux


3) S'assurer du type des valeurs de temperature_c et humidity_pct :
D'après le schéma de la df_weather:
temperature_c => est en str => KO
humidity_pct => float64     => OK

Conversion de temperature_c en float:

Les valeurs non convertible en float:

In [ ]:
df_weather.loc[
    pd.to_numeric(df_weather["temperature_c"], errors="coerce").isna(),
    "temperature_c"
].unique()[:10]

<StringArray>
['11,1', '1,7', '0,8', '11,6', '1,4', '2,7', '5,1', '12,5', '4,8', '12,7']
Length: 10, dtype: str

Solution pour convertir en float:

In [7]:
if "temperature_c" in df_weather.columns:
    # Convertir en string temporairement pour remplacer la virgule
    if df_weather["temperature_c"].astype(str).str.contains(",").any():
        df_weather["temperature_c"] = (
            df_weather["temperature_c"].astype(str)
            .str.replace(",", ".", regex=False)
            .astype(float)
        )


df_weather.info()

<class 'pandas.DataFrame'>
Index: 42172 entries, 6149 to 42171
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   city               42172 non-null  str           
 1   timestamp          10658 non-null  datetime64[us]
 2   temperature_c      41724 non-null  float64       
 3   humidity_pct       41760 non-null  float64       
 4   wind_speed_kmh     42172 non-null  float64       
 5   precipitation_mm   42172 non-null  float64       
 6   weather_condition  41310 non-null  str           
dtypes: datetime64[us](1), float64(4), str(2)
memory usage: 2.6 MB


3) Interpolation linéaire :
- Température :

In [8]:
df_weather["temperature_c"] = df_weather["temperature_c"].interpolate(method="linear")


- Humidité :

In [9]:
df_weather["humidity_pct"] = df_weather["humidity_pct"].interpolate(method="linear")

Vérifier qu'il n'y a plus de NaN dans les colonnes temperature_c & humidity_pct:

In [10]:
df_weather.isna().sum()

city                     0
timestamp            31514
temperature_c            0
humidity_pct             0
wind_speed_kmh           0
precipitation_mm         0
weather_condition      862
dtype: int64

==> Il n'y a plus de NaN dans ces 2 colonnes après l'interpolation.

  - Forward fill pour les conditions météo:
  (on a 862 NaN dans weather_condition)
  forward fill (ffill) : On prend la dernière valeur connue et on la recopie pour remplir les NaN suivants cat on suppose que la condition ne change pas brutalement entre deux mesures :

In [11]:
df_weather["weather_condition"] = df_weather["weather_condition"].ffill()


Vérifier qu'il n'y a plus de NaN dans la colonne weather_condition :

In [12]:
df_weather.isna().sum()

city                     0
timestamp            31514
temperature_c            0
humidity_pct             0
wind_speed_kmh           0
precipitation_mm         0
weather_condition        0
dtype: int64

==> Il n'y a plus de NaN dans cette colonne (weather_condition) après Forward fill.

### Corriger les valeurs aberrantes :
#### Températures hors [-40, 50] : remplacer par NaN puis interpoler:

In [17]:
## Remplacer les températures < -40 ou > 50 par NaN :
df_weather.loc[(df_weather["temperature_c"] < -40) | (df_weather["temperature_c"] > 50), "temperature_c"] = np.nan

Test si il y'a de nouveau NaN dans temperature_c :

In [20]:
df_weather["temperature_c"].isna().sum()  # combien de NaN maintenant
df_weather[df_weather["temperature_c"].isna()]    # afficher les lignes concernées

,city,timestamp,temperature_c,humidity_pct,wind_speed_kmh,precipitation_mm,weather_condition
14414,Lyon,2024-01-03 01:00:00,NaN,54.2,37.0,2.5,pluvieux
36012,Toulouse,2024-01-06 10:00:00,NaN,67.7,16.7,3.8,pluvieux
6519,Rouen,2024-01-06 16:00:00,NaN,69.9,21.8,0.0,brumeux
94,Paris,2024-01-07 04:00:00,NaN,77.5,28.8,5.3,orageux
16411,Lyon,2024-01-07 22:00:00,NaN,64.7,15.8,3.9,pluvieux
...,...,...,...,...,...,...,...
41564,Marseille,NaT,NaN,88.2,44.1,8.4,orageux
41689,Bordeaux,NaT,NaN,82.8,30.6,0.0,brumeux
41876,Strasbourg,NaT,NaN,67.6,9.5,0.0,nuageux
42046,Lyon,NaT,NaN,76.9,15.5,2.0,pluvieux


Interpolation:

In [ ]:
df_weather["temperature_c"] = df_weather["temperature_c"].interpolate(method="linear")

# test:
df_weather["temperature_c"].isna().sum()  # combien de NaN maintenant => 0

np.int64(0)

#### Humidité hors [0, 100] : clipper:

In [ ]:
df_weather["humidity_pct"] = df_weather["humidity_pct"].clip(lower=0, upper=100)
df_weather["humidity_pct"].min(), df_weather["humidity_pct"].max() # Vérification


(np.float64(40.0), np.float64(100.0))

### Standardiser les formats de dates:
Déjà fait pour l'intérpolation des valeurs de la colonne temperature_c.

### Fusionner avec les données de pollution (jointure sur ville et heure arrondie):